In [0]:
from pyspark.sql.functions import*
from pyspark.sql.types import*
from pyspark.sql.window import*

In [0]:
%sql
use catalog claims_leakage;
use schema gold;

In [0]:
fnol = spark.table("silver.fnol_clean")
policy = spark.table("silver.policy_clean_detailed")

risk_base = (
    fnol.alias("f")
    .join(
        policy.alias("p"),
        col("f.policy_id_canonical") == col("p.policy_id"),
        "left"
    )
)
display(risk_base)

In [0]:
gold_claim_risk_features = (
    risk_base
    .withColumn(
        "days_to_report",
        datediff(col("reported_date_ts"), col("loss_date_ts"))
    )

    .withColumn(
        "late_reporting_flag",
        when(col("days_to_report") > 7, 1).otherwise(0)
    )

    .withColumn(
        "high_fnol_amount_flag",
        when(col("claim_amount_num") > 0.8 * col("coverage_limit_num"), 1).otherwise(0)
    )

    .withColumn(
        "risky_loss_type_flag",
        when(col("loss_type_std").isin("THEFT", "BODILY_INJURY", "FIRE"), 1).otherwise(0)
    )

    .withColumn(
        "risky_geo_flag",
        when(col("incident_state_std").isin("MH", "KA", "DL"), 1).otherwise(0)
    )

    .withColumn(
        "digital_channel_flag",
        when(col("reporting_channel_std").isin("APP", "WEB"), 1).otherwise(0)
    )

    .select(
        "fnol_id",
        "policy_id_canonical",
        "days_to_report",
        "late_reporting_flag",
        "high_fnol_amount_flag",
        "risky_loss_type_flag",
        "risky_geo_flag",
        "digital_channel_flag"
    )
)
display(gold_claim_risk_features)

In [0]:
gold_claim_risk_features.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_claim_risk_features")


In [0]:
%sql
select * from gold_claim_risk_features

debug point

In [0]:
claims = spark.table("silver.claims_clean")

leakage_base = (
    claims.alias("c")
    .join(policy.alias("p"), col("c.policy_id") == col("p.policy_id"), "left")
    .join(fnol.alias("f"), col("c.fnol_id") == col("f.fnol_id"), "left")
)
display(leakage_base)

In [0]:
gold_claim_leakage_flags = (
    leakage_base
    .withColumn(
        "paid_gt_approved_flag",
        when(col("paid_amount_num") > col("approved_amount_num"), 1).otherwise(0)
    )

    .withColumn(
        "paid_gt_coverage_flag",
        when(col("paid_amount_num") > col("coverage_limit_num"), 1).otherwise(0)
    )

    .withColumn(
        "claim_after_policy_expiry_flag",
        when(
            ~col("f.loss_date_ts").between(
                col("p.policy_start_date_ts"),
                col("p.policy_end_date_ts")
            ),
            1
        ).otherwise(0)
    )

    .withColumn(
        "leakage_amount",
        greatest(
            col("paid_amount_num") - col("approved_amount_num"),
            lit(0)
        )
    )

    .select(
        col("c.claim_id").alias("claim_id"),
        col("c.fnol_id").alias("fnol_id"),
        col("p.policy_id").alias("policy_id"),
        "paid_gt_approved_flag",
        "paid_gt_coverage_flag",
        "claim_after_policy_expiry_flag",
        "leakage_amount"
    )

)
display(gold_claim_leakage_flags)

In [0]:
gold_claim_leakage_flags.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema","true")\
    .saveAsTable("gold_claim_leakage_flags")


In [0]:
risk = spark.table("gold_claim_risk_features")
leakage = spark.table("gold_claim_leakage_flags")

summary_base = (
    risk.alias("r")
    .join(leakage.alias("l"), "fnol_id", "left")
)
display(summary_base)

In [0]:
gold_claim_risk_summary = (
    summary_base
    .withColumn(
        "risk_score",
        coalesce(col("late_reporting_flag"), lit(0)) * 30 +
        coalesce(col("high_fnol_amount_flag"), lit(0)) * 25 +
        coalesce(col("risky_loss_type_flag"), lit(0)) * 20 +
        coalesce(col("risky_geo_flag"), lit(0)) * 15 +
        coalesce(col("paid_gt_approved_flag"), lit(0)) * 40 +
        coalesce(col("paid_gt_coverage_flag"), lit(0)) * 40
    )
    .withColumn(
        "risk_level",
        when(col("risk_score") >= 70, "HIGH")
        .when(col("risk_score") >= 40, "MEDIUM")
        .otherwise("LOW")
    )
    .withColumn(
        "leakage_amount",
        coalesce(col("leakage_amount"), lit(0))
    )
    .withColumn(
        "leakage_flag",
        when(col("leakage_amount") > 0, "YES").otherwise("NO")
    )
    .withColumn(
    "risk_reasons",
    concat_ws(
        ", ",
        when(col("late_reporting_flag") == 1, "Late reporting"),
        when(col("high_fnol_amount_flag") == 1, "High FNOL amount"),
        when(col("risky_loss_type_flag") == 1, "Risky loss type"),
        when(col("risky_geo_flag") == 1, "Risky geography"),
        when(col("paid_gt_approved_flag") == 1, "Paid > approved"),
        when(col("paid_gt_coverage_flag") == 1, "Paid > coverage")
    )
)




    .select(
        "claim_id",
        "fnol_id",
        "policy_id_canonical",
        "risk_score",
        "risk_level",
        "leakage_flag",
        "leakage_amount",
        "risk_reasons"
    )
)
display(gold_claim_risk_summary)


In [0]:
gold_claim_risk_summary.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_claim_risk_summary")

In [0]:
%sql
SELECT *
                    FROM claims_leakage.gold.gold_claim_risk_summary
                    WHERE leakage_flag !="NO"
                    
                    